In [48]:
from math import gamma, sqrt, pi
from scipy.stats import t
import numpy as np

import plotly.express as px
import plotly.figure_factory as ff

import matplotlib.pylab as plt
import seaborn as sns

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd

from ipywidgets import FloatSlider, IntSlider, VBox, HBox, Checkbox

import ipydatagrid

# Skewed Student-t Inverse CDF
<img src="Images/skt_definitions.png"/>
<img  src="Images/skt_inv.png"/>

In [49]:
def sktinv(u,nu,_lambda):
    '''
    % sktinv(u,nu,_lambda)
    % u      = Cdf value (Between 0 and 1)
    % nu     = Kurtosis Parameter (>2 to inf)
    % lambda = Assymetry Parameter (-1 to 1)
    %
    % returns z = random Variable (between -inf and inf)
    %
    % Calculates the inverse CDF for the skewed Student-t. Hansen (1994) version.
    % Author: Tino Contino
    % Date:   2 May 2021
    '''
    
    u  = np.atleast_1d(u)
    
    c = gamma((nu+1)/2)/(sqrt(pi*(nu-2))*gamma(nu/2))
    a = 4*_lambda*c*((nu-2)/(nu-1))
    b = sqrt(1 + 3 * _lambda**2 - a**2)
    
    inv1 = (1-_lambda)/b * sqrt((nu-2) / nu) * t.ppf(u / (1-_lambda), nu) - a/b
    inv2 = (1+_lambda)/b * sqrt((nu-2) / nu) * t.ppf(0.5 + (1 / (1+_lambda)) * (u - (1-_lambda)/2) , nu) - a/b
    
    limit_variable = (1-_lambda)/2
    
    z = u.copy()
    
    lt = u < limit_variable
    gt = u >= limit_variable
    
    z[lt] = inv1[lt]
    z[gt] = inv2[gt]

    return z

# Skewed Student-t CDF
<img src="Images/skt_cdf.png"/>

In [50]:
def sktcdf(z,nu,_lambda):
    '''
    % sktcdf(z,nu,lambda)
    % z      = Random Variable value (Between -inf and inf)
    % nu     = Kurtosis Parameter (>2 to inf)
    % lambda = Assymetry Parameter (-1 to 1)
    %
    % returns u = random Variable (between 0 and 1)
    %
    % Calculates the CDF for the skewed Student-t. Hansen (1994) version.
    % Author: Tino Contino
    % Date:   2 May 2021
    '''
    
    z  = np.atleast_1d(z)
    
    c = gamma((nu+1)/2)/(sqrt(pi*(nu-2))*gamma(nu/2))
    a = 4*_lambda*c*((nu-2)/(nu-1))
    b = sqrt(1 + 3 * _lambda**2 - a**2)
    
    limit_variable = -a/b
    lt = z < limit_variable
    gt = z >= limit_variable
    
    y_1 = (b*z+a) / (1-_lambda) * sqrt(nu/(nu-2))
    y_2 = (b*z+a) / (1+_lambda) * sqrt(nu/(nu-2))
    
    pdf1 = (1-_lambda) * t.cdf(y_1, nu)
    pdf2 = (1-_lambda)/2 + (1+_lambda) * (t.cdf(y_2, nu)-0.5)

    u = z.copy()

    u[lt] = pdf1[lt]
    u[gt] = pdf2[gt]

    return u

# Skewed Student-t PDF
<img src="Images/skt_pdf.png"/>

In [136]:
 def sktpdf(z, nu, _lambda):
    '''
    % sktpdf(z,nu,lambda)
    % z      = Random Variable value (Between -inf and inf)
    % nu     = Kurtosis Parameter (>2 to inf)
    % lambda = Assymetry Parameter (-1 to 1)
    %
    % returns u = random Variable > 0
    %
    % Calculates the PDF for the skewed Student-t. Hansen (1994) version.
    % Author: Tino Contino
    % Date:   27 May 2021
    '''

    z  = np.atleast_1d(z)

    c = gamma((nu+1)/2)/(sqrt(pi*(nu-2))*gamma(nu/2))
    a = 4*_lambda*c*((nu-2)/(nu-1))
    b = sqrt(1 + 3 * _lambda**2 - a**2)

    limit_variable = -a/b
    lt = z < limit_variable
    gt = z >= limit_variable

    y_1 = b * c * (1 + (1/(nu-2)) * ((b*z+a) / (1-_lambda))**2 )**-((nu+1)/2)
    y_2 = b * c * (1 + (1/(nu-2)) * ((b*z+a) / (1+_lambda))**2 )**-((nu+1)/2)

    u = z.copy()

    u[lt] = y_1[lt]
    u[gt] = y_2[gt]
    return u

In [137]:
def update_plots(change):
    scatter = f.data[0]
    scatter.y = sktinv(u,nu = nu_slider.value, _lambda = lambda_slider.value)
    f.layout.title = f'Inverse CDF Skewed Student-t, Nu = {nu_slider.value:.2f} Lambda = {lambda_slider.value:.2f}'
    
    scatter = g.data[0]
    scatter.y = sktcdf(z,nu = nu_slider.value, _lambda = lambda_slider.value)
    g.layout.title = f'CDF Skewed Student-t, Nu = {(nu_slider.value):.2f} Lambda = {(lambda_slider.value):.2f}'
    
    scatter = h.data[0]
    scatter.y = sktpdf(z,nu = nu_slider.value, _lambda = lambda_slider.value)
    update_grid()
    if show_student_t_checkbox.value:
        scale = (nu_slider.value/(nu_slider.value-2))**.5
        
        scatter_t = f.data[1]
        scatter_t.y = t.ppf(u, nu_slider.value, scale=1/scale)
        
        scatter_t = g.data[1]
        scatter_t.y = t.cdf(z, nu_slider.value, scale=1/scale)
        
        scatter_t = h.data[1]
        scatter_t.y = t.pdf(z, nu_slider.value, scale=1/scale)
        
    h.layout.title = f'PDF Skewed Student-t, Nu = {(nu_slider.value):.2f} Lambda = {(lambda_slider.value):.2f}'

In [138]:
def plot_student_t(change):
    if ~show_student_t_checkbox.value:
        f.data = [f.data[0]]
        g.data = [g.data[0]]
        h.data = [h.data[0]]
        update_grid()
        
    if change['new']:
        scale = (nu_slider.value/(nu_slider.value-2))**.5
        f.add_scatter(x=u, y = t.ppf(u, nu_slider.value, scale=1/scale), name = 'Student-t')
        g.add_scatter(x=z, y = t.cdf(z, nu_slider.value, scale=1/scale), name = 'Student-t')
        h.add_scatter(x=z, y = t.pdf(z, nu_slider.value, scale=1/scale), name = 'Student-t')
        update_grid()

In [139]:
def update_grid():
    df = pd.DataFrame(columns = ['Skew-t'], data = np.random.rand(4,1), index=  ['Mean', 'Variance', 'Skew', 'Kurtosis'])
    nu = nu_slider.value
    _lambda = lambda_slider.value
    c = gamma((nu+1)/2)/(sqrt(pi*(nu-2))*gamma(nu/2))
    a = 4*_lambda*c*((nu-2)/(nu-1))
    b = sqrt(1 + 3 * _lambda**2 - a**2)
    
    m2 = 1 + 3 * _lambda**2
    
    df.loc['Mean', 'Skew-t'] = 0
    df.loc['Variance', 'Skew-t'] = 1  
    if nu > 3:
        m3 = 16 * c * _lambda * (1+_lambda**2) * (nu-2)**2 / ((nu-1)*(nu-3))
        df.loc['Skew', 'Skew-t'] = (m3 - 3*a*m2 + 2*a**3) / b**3
    else:
        df.loc['Skew', 'Skew-t'] = np.nan
    if nu > 4:
        m4 = 3 * (nu-2)/(nu-4) * (1+10*_lambda**2 + 5*_lambda**4)
        df.loc['Kurtosis', 'Skew-t'] = (m4 -4*a*m3 + 6*a**2 * m3 - 3*a**4) / b**4
    else:
        df.loc['Kurtosis', 'Skew-t'] = np.nan
    if show_student_t_checkbox.value:
        df.loc['Mean', 'Student-t'] = 0
        df.loc['Variance', 'Student-t'] = 1
        if nu > 3:
            df.loc['Skew', 'Student-t'] = 0
        else:
            df.loc['Skew', 'Student-t'] = np.nan
        if nu > 4:
            df.loc['Kurtosis', 'Student-t'] = 3 + 6 / (nu-4)
        else:
            df.loc['Kurtosis', 'Student-t'] = np.nan
    grid.data = df.round(2)

In [156]:
nu_slider = FloatSlider(min=2.1, max=40.0, step=0.1, value=4, description = 'Nu', continuous_update=False)
lambda_slider = FloatSlider(min=-0.99, max=0.99, step=0.01, value=0, description = 'Lambda', continuous_update=False)
show_student_t_checkbox = Checkbox(description = 'Show Student-t')

nu_slider.observe(update_plots, names='value')
lambda_slider.observe(update_plots, names='value')
show_student_t_checkbox.observe(plot_student_t, names='value')

df = pd.DataFrame(columns = ['Skew-t'], data = np.random.rand(4,1), index=  ['Mean', 'Variance', 'Skew', 'Kurtosis'])

nu = nu_slider.value
_lambda = lambda_slider.value

c = gamma((nu+1)/2)/(sqrt(pi*(nu-2))*gamma(nu/2))
a = 4*_lambda*c*((nu-2)/(nu-1))
b = sqrt(1 + 3 * _lambda**2 - a**2)

m2 = 1 + 3 * _lambda**2

df.loc['Mean', 'Skew-t'] = 0
df.loc['Variance', 'Skew-t'] = 1  
if nu > 3:
    m3 = 16 * c * _lambda * (1+_lambda**2) * (nu-2)**2 / ((nu-1)*(nu-3))
    df.loc['Skew', 'Skew-t'] = (m3 - 3*a*m2 + 2*a**3) / b**3
else:
    df.loc['Skew', 'Skew-t'] = np.nan
if nu > 4:
    m4 = 3 * (nu-2)/(nu-4) * (1+10*_lambda**2 + 5*_lambda**4)
    df.loc['Kurtosis', 'Skew-t'] = (m4 -4*a*m3 + 6*a**2 * m3 - 3*a**4) / b**4
else:
    df.loc['Kurtosis', 'Skew-t'] = np.nan

grid = ipydatagrid.DataGrid(df, layout= {'width' : '400px'}, base_column_size = 150)

In [157]:
nu = nu_slider.value
_lambda = lambda_slider.value
u = np.linspace(.01, .99, 100)
z = np.linspace(-3, 3, 100)

f = go.FigureWidget()
f.add_scatter(x = u, y=sktinv(u,nu = nu, _lambda = _lambda), name = 'Skewed-t');
f.layout.title = f'Inverse CDF Skewed Student-t, Nu = {nu_slider.value:.2f} Lambda = {lambda_slider.value:.2f}'

In [158]:
g = go.FigureWidget()
g.add_scatter(x = z, y=sktcdf(z,nu = nu, _lambda = _lambda), name = 'Skewed-t');
g.layout.title = f'CDF Skewed Student-t, Nu = {(nu_slider.value):.2f} Lambda = {(lambda_slider.value):.2f}'

In [159]:
h = go.FigureWidget()
h.add_scatter(x = z, y=sktpdf(z, nu = nu_slider.value, _lambda = lambda_slider.value), name = 'Skewed-t')
h.layout.title = f'PDF Skewed Student-t, Nu = {(nu_slider.value):.2f} Lambda = {(lambda_slider.value):.2f}'

In [160]:
VBox([HBox([f, g]),
      HBox([h, VBox([nu_slider,
          lambda_slider,
         show_student_t_checkbox,
         grid])])
           ])

    'data': [{'name': 'Skewed-t',
              'type': 'scatter'…

In [145]:
u = np.random.uniform(size=(1000,1))

In [25]:
sktpdf(u,200,0.2)

array([[0.37697852],
       [0.32733246],
       [0.29651144],
       [0.36818705],
       [0.37498825],
       [0.31406242],
       [0.36422063],
       [0.35233657],
       [0.3728643 ],
       [0.28563604],
       [0.31045201],
       [0.38680797],
       [0.31483176],
       [0.38507901],
       [0.36883331],
       [0.35404066],
       [0.32536749],
       [0.28393958],
       [0.26139868],
       [0.27620005],
       [0.32122646],
       [0.34665722],
       [0.23251142],
       [0.37353528],
       [0.3763146 ],
       [0.38728702],
       [0.33733737],
       [0.2860554 ],
       [0.37127237],
       [0.36337137],
       [0.30718879],
       [0.33634635],
       [0.30680959],
       [0.33365448],
       [0.30389845],
       [0.32065429],
       [0.31277495],
       [0.26877585],
       [0.34144044],
       [0.35505572],
       [0.23804958],
       [0.29738342],
       [0.22014463],
       [0.35000668],
       [0.37406905],
       [0.24368218],
       [0.35852956],
       [0.371

In [28]:
sktinv(u,5,0.3)

array([[-6.60623386e-01],
       [-6.84111308e-01],
       [-7.04073574e-01],
       [-1.00766041e+00],
       [-7.95918151e-01],
       [-4.43511297e-01],
       [-3.41030594e-01],
       [-4.75449252e+00],
       [-1.09978658e+00],
       [ 2.56629964e-02],
       [ 1.94303392e-01],
       [-3.77287590e-01],
       [ 1.19470649e+00],
       [-1.32904777e+00],
       [ 4.81803008e-01],
       [-6.72427044e-01],
       [ 7.74013900e-01],
       [ 6.06480513e-01],
       [-5.55941262e-01],
       [-6.84537876e-01],
       [ 1.67003794e+00],
       [-8.45360326e-01],
       [-3.95905050e-02],
       [ 1.98737284e-01],
       [-5.76443029e-01],
       [ 2.33267648e-01],
       [ 8.01040908e-01],
       [ 7.41503756e-01],
       [-6.67698327e-01],
       [ 2.13561755e-01],
       [ 2.36096238e-01],
       [ 4.31002741e-02],
       [-1.69582586e+00],
       [-1.04763867e+00],
       [ 4.80218839e-01],
       [-2.10641760e-02],
       [-7.85292878e-01],
       [-7.46731541e-01],
       [-4.7